# Import Package

In [2]:
from random import randint
from time import time
import numpy as np
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

Using TensorFlow backend.


# Load Data

In [2]:
def load_data(path='cube_dataset.npz'):
    f = np.load(path)
    x_train, y_train, x_test, y_test = f['X_train'], f['y_train'], f['X_test'], f['y_test']
    f.close()
    return (x_train, y_train, x_test, y_test)

(X_train, y_train, X_test, y_test) = load_data('./dataset/ds_10000_10000_10_18.npz')
print(X_train.shape)

(100000, 18, 3, 6)


# Set Constant

In [4]:
NB_EPOCH = 12000
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
VERBOSE = 1
INPUT_SHAPE = (18, 3, 6)
NUM_CLASSES = 18
MODEL = "1Conv3dense_no_onehot_12000_32_18_f"
MODEL_DIR = os.path.join("./model",MODEL)
CKP_DIR = os.path.join("./ckp_tmp",MODEL)
TBLOG_DIR = os.path.join("./tb_log",MODEL,"{}")
if not os.path.exists(CKP_DIR):
    os.mkdir(CKP_DIR)

# Batch Generator

In [4]:
def batch_generator(X_train = X_train, y_train = y_train, batch_size = BATCH_SIZE):
    while True:
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        X = X_train[idx]
        y = y_train[idx]
        yield (X,y)

# Construct Neural Network

In [7]:
model = Sequential()
model.add(Conv2D(512, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE))

model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 16, 1, 512)        28160     
_________________________________________________________________
dense_13 (Dense)             (None, 16, 1, 1024)       525312    
_________________________________________________________________
dense_14 (Dense)             (None, 16, 1, 512)        524800    
_________________________________________________________________
dense_15 (Dense)             (None, 16, 1, 512)        262656    
_________________________________________________________________
dense_16 (Dense)             (None, 16, 1, 512)        262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 1, 512)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
__________

In [6]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
tensorboard = TensorBoard(log_dir=TBLOG_DIR.format(time()), write_graph=False)
checkpoint = ModelCheckpoint(filepath=os.path.join(CKP_DIR, "model-{epoch:02d}.h5"), save_best_only=True)

# Train Model

In [7]:
model.fit_generator(generator=batch_generator(),
                    steps_per_epoch=1,
                    epochs=NB_EPOCH,
                    verbose=VERBOSE,
                    validation_data=batch_generator(X_test, y_test),
                    validation_steps=1,
                    callbacks=[tensorboard, checkpoint])

Epoch 1/12000
1/1 [==============================] - 1s 604ms/step - loss: 2.8883 - acc: 0.0625 - val_loss: 2.8882 - val_acc: 0.0312
Epoch 2/12000
1/1 [==============================] - 1s 614ms/step - loss: 2.8905 - acc: 0.0625 - val_loss: 2.8863 - val_acc: 0.0625
Epoch 3/12000
1/1 [==============================] - 0s 437ms/step - loss: 2.8939 - acc: 0.0312 - val_loss: 2.8897 - val_acc: 0.0625
Epoch 4/12000
1/1 [==============================] - 0s 485ms/step - loss: 2.8813 - acc: 0.0938 - val_loss: 2.8865 - val_acc: 0.0312
Epoch 5/12000
1/1 [==============================] - 0s 466ms/step - loss: 2.8781 - acc: 0.0312 - val_loss: 2.8872 - val_acc: 0.0312
Epoch 6/12000
1/1 [==============================] - 1s 545ms/step - loss: 2.8978 - acc: 0.0312 - val_loss: 2.8887 - val_acc: 0.1250
Epoch 7/12000
1/1 [==============================] - 1s 588ms/step - loss: 2.8725 - acc: 0.0312 - val_loss: 2.8895 - val_acc: 0.0625
Epoch 8/12000
1/1 [==============================] - 0s 457ms/step - 

Epoch 62/12000
1/1 [==============================] - 0s 442ms/step - loss: 2.8389 - acc: 0.0938 - val_loss: 2.7643 - val_acc: 0.0000e+00
Epoch 63/12000
1/1 [==============================] - 0s 441ms/step - loss: 2.7351 - acc: 0.2500 - val_loss: 2.8664 - val_acc: 0.0000e+00
Epoch 64/12000
1/1 [==============================] - 0s 433ms/step - loss: 2.7275 - acc: 0.0625 - val_loss: 2.7133 - val_acc: 0.1875
Epoch 65/12000
1/1 [==============================] - 0s 437ms/step - loss: 2.8218 - acc: 0.0938 - val_loss: 2.7615 - val_acc: 0.3125
Epoch 66/12000
1/1 [==============================] - 0s 439ms/step - loss: 2.8181 - acc: 0.1250 - val_loss: 2.8467 - val_acc: 0.0625
Epoch 67/12000
1/1 [==============================] - 0s 423ms/step - loss: 2.7963 - acc: 0.0312 - val_loss: 2.7716 - val_acc: 0.2188
Epoch 68/12000
1/1 [==============================] - 0s 467ms/step - loss: 2.8477 - acc: 0.0938 - val_loss: 2.6508 - val_acc: 0.2812
Epoch 69/12000
1/1 [==============================] - 

Epoch 123/12000
1/1 [==============================] - 0s 431ms/step - loss: 2.3511 - acc: 0.3438 - val_loss: 2.1060 - val_acc: 0.3750
Epoch 124/12000
1/1 [==============================] - 0s 450ms/step - loss: 2.1830 - acc: 0.4062 - val_loss: 2.2932 - val_acc: 0.2500
Epoch 125/12000
1/1 [==============================] - 0s 452ms/step - loss: 2.3228 - acc: 0.3125 - val_loss: 2.3052 - val_acc: 0.3438
Epoch 126/12000
1/1 [==============================] - 0s 452ms/step - loss: 2.2415 - acc: 0.3125 - val_loss: 2.4368 - val_acc: 0.2812
Epoch 127/12000
1/1 [==============================] - 0s 434ms/step - loss: 2.4024 - acc: 0.2500 - val_loss: 2.2879 - val_acc: 0.3125
Epoch 128/12000
1/1 [==============================] - 0s 425ms/step - loss: 2.1435 - acc: 0.4375 - val_loss: 2.4847 - val_acc: 0.3750
Epoch 129/12000
1/1 [==============================] - 0s 437ms/step - loss: 2.2240 - acc: 0.4062 - val_loss: 1.9816 - val_acc: 0.3438
Epoch 130/12000
1/1 [==============================] - 

Epoch 184/12000
1/1 [==============================] - 0s 449ms/step - loss: 2.4113 - acc: 0.2812 - val_loss: 2.1610 - val_acc: 0.3438
Epoch 185/12000
1/1 [==============================] - 0s 428ms/step - loss: 2.2794 - acc: 0.4375 - val_loss: 2.5175 - val_acc: 0.3125
Epoch 186/12000
1/1 [==============================] - 0s 429ms/step - loss: 2.4075 - acc: 0.2812 - val_loss: 2.1949 - val_acc: 0.2812
Epoch 187/12000
1/1 [==============================] - 0s 451ms/step - loss: 2.4610 - acc: 0.2500 - val_loss: 2.3151 - val_acc: 0.3750
Epoch 188/12000
1/1 [==============================] - 0s 434ms/step - loss: 1.9779 - acc: 0.4375 - val_loss: 2.4153 - val_acc: 0.3125
Epoch 189/12000
1/1 [==============================] - 0s 449ms/step - loss: 2.0757 - acc: 0.4375 - val_loss: 2.5294 - val_acc: 0.2188
Epoch 190/12000
1/1 [==============================] - 0s 420ms/step - loss: 2.0543 - acc: 0.3750 - val_loss: 2.1021 - val_acc: 0.4062
Epoch 191/12000
1/1 [==============================] - 

Epoch 245/12000
1/1 [==============================] - 0s 447ms/step - loss: 2.1106 - acc: 0.3750 - val_loss: 2.3235 - val_acc: 0.2188
Epoch 246/12000
1/1 [==============================] - 0s 446ms/step - loss: 2.0287 - acc: 0.3750 - val_loss: 2.2419 - val_acc: 0.3750
Epoch 247/12000
1/1 [==============================] - 0s 452ms/step - loss: 2.0346 - acc: 0.4062 - val_loss: 2.0513 - val_acc: 0.4375
Epoch 248/12000
1/1 [==============================] - 0s 441ms/step - loss: 2.5332 - acc: 0.2500 - val_loss: 1.9741 - val_acc: 0.3438
Epoch 249/12000
1/1 [==============================] - 0s 450ms/step - loss: 1.9998 - acc: 0.4688 - val_loss: 1.8277 - val_acc: 0.5000
Epoch 250/12000
1/1 [==============================] - 0s 453ms/step - loss: 2.1679 - acc: 0.3750 - val_loss: 2.1407 - val_acc: 0.4375
Epoch 251/12000
1/1 [==============================] - 0s 439ms/step - loss: 2.2576 - acc: 0.3125 - val_loss: 2.1750 - val_acc: 0.3750
Epoch 252/12000
1/1 [==============================] - 

...

Epoch 11745/12000
1/1 [==============================] - 1s 721ms/step - loss: 1.2339 - acc: 0.6875 - val_loss: 2.0569 - val_acc: 0.4688
Epoch 11746/12000
1/1 [==============================] - 1s 654ms/step - loss: 1.8545 - acc: 0.4688 - val_loss: 1.7139 - val_acc: 0.5000
Epoch 11747/12000
1/1 [==============================] - 1s 613ms/step - loss: 1.9988 - acc: 0.5000 - val_loss: 1.4809 - val_acc: 0.5938
Epoch 11748/12000
1/1 [==============================] - 1s 644ms/step - loss: 1.2956 - acc: 0.6250 - val_loss: 2.1579 - val_acc: 0.4062
Epoch 11749/12000
1/1 [==============================] - 1s 609ms/step - loss: 1.3732 - acc: 0.5625 - val_loss: 1.3461 - val_acc: 0.5000
Epoch 11750/12000
1/1 [==============================] - 1s 654ms/step - loss: 1.4089 - acc: 0.6562 - val_loss: 1.2811 - val_acc: 0.5625
Epoch 11751/12000
1/1 [==============================] - 1s 622ms/step - loss: 1.5500 - acc: 0.5312 - val_loss: 1.9815 - val_acc: 0.4688
Epoch 11752/12000
1/1 [==================

Epoch 11805/12000
1/1 [==============================] - 1s 579ms/step - loss: 1.2310 - acc: 0.5625 - val_loss: 1.8428 - val_acc: 0.4375
Epoch 11806/12000
1/1 [==============================] - 1s 571ms/step - loss: 1.1603 - acc: 0.5938 - val_loss: 2.0153 - val_acc: 0.4062
Epoch 11807/12000
1/1 [==============================] - 1s 572ms/step - loss: 1.6941 - acc: 0.5000 - val_loss: 1.7135 - val_acc: 0.4688
Epoch 11808/12000
1/1 [==============================] - 1s 603ms/step - loss: 1.4732 - acc: 0.5625 - val_loss: 1.4915 - val_acc: 0.4688
Epoch 11809/12000
1/1 [==============================] - 1s 594ms/step - loss: 1.6460 - acc: 0.5000 - val_loss: 1.4313 - val_acc: 0.5938
Epoch 11810/12000
1/1 [==============================] - 1s 616ms/step - loss: 1.2596 - acc: 0.6562 - val_loss: 1.9036 - val_acc: 0.4688
Epoch 11811/12000
1/1 [==============================] - 1s 634ms/step - loss: 1.4480 - acc: 0.5625 - val_loss: 2.1887 - val_acc: 0.3750
Epoch 11812/12000
1/1 [==================

Epoch 11865/12000
1/1 [==============================] - 0s 438ms/step - loss: 1.6751 - acc: 0.6250 - val_loss: 1.6820 - val_acc: 0.5938
Epoch 11866/12000
1/1 [==============================] - 0s 441ms/step - loss: 1.6091 - acc: 0.5312 - val_loss: 1.6977 - val_acc: 0.5000
Epoch 11867/12000
1/1 [==============================] - 0s 439ms/step - loss: 1.6629 - acc: 0.4688 - val_loss: 1.8411 - val_acc: 0.5625
Epoch 11868/12000
1/1 [==============================] - 0s 450ms/step - loss: 1.6319 - acc: 0.5625 - val_loss: 1.3773 - val_acc: 0.5938
Epoch 11869/12000
1/1 [==============================] - 0s 430ms/step - loss: 1.4278 - acc: 0.5938 - val_loss: 1.6679 - val_acc: 0.4688
Epoch 11870/12000
1/1 [==============================] - 0s 447ms/step - loss: 1.2457 - acc: 0.6875 - val_loss: 1.2661 - val_acc: 0.5625
Epoch 11871/12000
1/1 [==============================] - 0s 458ms/step - loss: 1.3847 - acc: 0.5938 - val_loss: 1.5662 - val_acc: 0.6562
Epoch 11872/12000
1/1 [==================

Epoch 11925/12000
1/1 [==============================] - 1s 700ms/step - loss: 2.0766 - acc: 0.5312 - val_loss: 2.0323 - val_acc: 0.4062
Epoch 11926/12000
1/1 [==============================] - 1s 697ms/step - loss: 1.4786 - acc: 0.5625 - val_loss: 2.0382 - val_acc: 0.4062
Epoch 11927/12000
1/1 [==============================] - 1s 745ms/step - loss: 2.1903 - acc: 0.4688 - val_loss: 1.4182 - val_acc: 0.5938
Epoch 11928/12000
1/1 [==============================] - 1s 722ms/step - loss: 1.9830 - acc: 0.4062 - val_loss: 1.7015 - val_acc: 0.5938
Epoch 11929/12000
1/1 [==============================] - 1s 671ms/step - loss: 1.3935 - acc: 0.5625 - val_loss: 1.9654 - val_acc: 0.3750
Epoch 11930/12000
1/1 [==============================] - 1s 589ms/step - loss: 1.3156 - acc: 0.6562 - val_loss: 1.4954 - val_acc: 0.5000
Epoch 11931/12000
1/1 [==============================] - 1s 593ms/step - loss: 1.2174 - acc: 0.7500 - val_loss: 1.8411 - val_acc: 0.3438
Epoch 11932/12000
1/1 [==================

Epoch 11985/12000
1/1 [==============================] - 1s 631ms/step - loss: 1.0385 - acc: 0.5938 - val_loss: 2.2409 - val_acc: 0.5625
Epoch 11986/12000
1/1 [==============================] - -73s -72894161us/step - loss: 1.4738 - acc: 0.6250 - val_loss: 2.0888 - val_acc: 0.4375
Epoch 11987/12000
1/1 [==============================] - 1s 809ms/step - loss: 1.5482 - acc: 0.5625 - val_loss: 1.4044 - val_acc: 0.6562
Epoch 11988/12000
1/1 [==============================] - 1s 900ms/step - loss: 1.7078 - acc: 0.5000 - val_loss: 1.1841 - val_acc: 0.6562
Epoch 11989/12000
1/1 [==============================] - 1s 925ms/step - loss: 1.8905 - acc: 0.5312 - val_loss: 1.7178 - val_acc: 0.5312
Epoch 11990/12000
1/1 [==============================] - 1s 821ms/step - loss: 1.5308 - acc: 0.5625 - val_loss: 1.5108 - val_acc: 0.5312
Epoch 11991/12000
1/1 [==============================] - 1s 561ms/step - loss: 1.9562 - acc: 0.4375 - val_loss: 2.4832 - val_acc: 0.4062
Epoch 11992/12000
1/1 [==========

# Evaluate Model

In [8]:
score = model.evaluate_generator(batch_generator(X_test, y_test, 5000), steps=5, max_queue_size=10, workers=1, use_multiprocessing=False)
print("Test score:", score[0])
print("Test accuracy:", score[1])
model.save(MODEL_DIR+'.h5')

Test score: 1.70549254417
Test accuracy: 0.517959988117
